In [1]:
from numpy import *

In [2]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [3]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [11]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        #print(transaction)
        for item in transaction:
            #print(item)
            if not [item] in C1:
                C1.append([item])
                #print(C1)
                
    C1.sort()
    return list(map(frozenset, C1))

In [12]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [15]:
def scanD(dataSet, Ck, minSupport):
    ssCnt = {}
    for tid in dataSet:
        for can in Ck:
            if can.issubset(tid): #判断can是否是tid的子集
                if can not in ssCnt.keys(): 
                    ssCnt[can]=1
                else: 
                    ssCnt[can] += 1
    numItems = float(len(dataSet))
    retList = []      #频繁项集
    supportData = {}  #候选集项Ck的支持度（k：候选项，value：支持度）
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [17]:
L1, supportData = scanD(dataSet, C1, 0.5)

In [18]:
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [19]:
def aprioriGen(Lk, k): #Lk:频繁项集， k=项的个数
    CList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2: #如果L1和L2的前k-2个元素相同
                CList.append(Lk[i] | Lk[j]) #对其直接做并集处理
    return CList

In [21]:
C2 = aprioriGen(L1, 2)

In [22]:
L2, supportData = scanD(dataSet, C2, 0.5)

In [23]:
L2

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [24]:
C3 = aprioriGen(L2, 3)
C3

[frozenset({2, 3, 5})]

In [25]:
L3, supportData = scanD(dataSet, C3, 0.5)
L3

[frozenset({2, 3, 5})]

In [27]:
C4 = aprioriGen(L3, 4)
len(C4)

0

In [28]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    L1, supportData = scanD(dataSet, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(dataSet, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [29]:
L, supportData = apriori(dataSet, minSupport = 0.5)

In [30]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [31]:
supportData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}